# Test Runloop Backend Implementation

This notebook tests the RunloopProtocol backend implementation by creating a devbox and testing all methods.

## Setup: Create Runloop Client and Devbox

In [1]:
from deepagents.integrations.runloop import RunloopBackend
from runloop_api_client import Runloop

In [2]:
# Create Runloop client (API Key is automatically loaded from "RUNLOOP_API_KEY" environment variable)
client = Runloop()

# Create a simple devbox without code mounts
devbox = client.devboxes.create()
print(f"Devbox created with ID: {devbox.id}")

Devbox created with ID: dbx_31WKMQDRNkMz7eu3kidKr


In [3]:
# Instantiate the RunloopProtocol backend
backend = RunloopBackend(devbox_id=devbox.id, client=client)
print("RunloopProtocol backend instantiated successfully")

RunloopProtocol backend instantiated successfully


## Test 1: exec() - Execute Commands

In [5]:
# Test basic command execution
stdout, exit_code = backend.exec("echo 'Hello from Runloop!'")
print(f"Exit code: {exit_code}")
print(f"Output: {stdout}")

Exit code: 0
Output: Hello from Runloop!



In [6]:
# Test pwd and ls
stdout, exit_code = backend.exec("pwd")
print(f"Current directory: {stdout}")

stdout, exit_code = backend.exec("ls -la")
print(f"Directory listing:\n{stdout}")

Current directory: /home/user

Directory listing:
total 24
drwx------ 3 user user 4096 Nov  5 14:59 .
drwxr-xr-x 3 root root 4096 Sep 25 23:13 ..
-rw-r--r-- 1 user user  220 Sep 25 23:13 .bash_logout
-rw-r--r-- 1 user user 3526 Sep 25 23:13 .bashrc
-rw-r--r-- 1 user user  807 Sep 25 23:13 .profile
drwx------ 2 user user 4096 Nov  5 14:59 .ssh
-rw-r--r-- 1 user user    0 Nov  5 14:59 .sudo_as_admin_successful



## Test 2: write() - Create Files

In [7]:
# Create a test directory
backend.exec("mkdir -p /tmp/test_backend")

# Write a simple text file
result = backend.write(file_path="/tmp/test_backend/hello.txt", content="Hello World!\nThis is a test file.\nLine 3.")
print(f"Write result: {result}")

Write result: WriteResult(error=None, path='/tmp/test_backend/hello.txt', files_update=None)


In [8]:
# Write a Python file
result = backend.write(
    file_path="/tmp/test_backend/test.py",
    content="""def greet(name):
    return f'Hello, {name}!'

if __name__ == '__main__':
    print(greet('World'))
""",
)
print(f"Write result: {result}")

Write result: WriteResult(error=None, path='/tmp/test_backend/test.py', files_update=None)


In [9]:
# Test writing to existing file (should fail)
result = backend.write(file_path="/tmp/test_backend/hello.txt", content="This should fail")
print(f"Expected error: {result}")

Expected error: WriteResult(error='Cannot write to /tmp/test_backend/hello.txt because it already exists. Read and then make an edit, or write to a new path.', path=None, files_update=None)


## Test 3: read() - Read Files

In [10]:
# Read the text file
content = backend.read("/tmp/test_backend/hello.txt")
print("File content:")
print(content)

File content:
     1	Hello World!
     2	This is a test file.
     3	Line 3.


In [11]:
# Read with offset
content = backend.read("/tmp/test_backend/hello.txt", offset=1, limit=1)
print("File content with offset=1, limit=1:")
print(content)

File content with offset=1, limit=1:
     2	This is a test file.


In [12]:
# Read the Python file
content = backend.read("/tmp/test_backend/test.py")
print("Python file content:")
print(content)

Python file content:
     1	def greet(name):
     2	    return f'Hello, {name}!'
     3	
     4	if __name__ == '__main__':
     5	    print(greet('World'))


In [13]:
# Read non-existent file
content = backend.read("/tmp/test_backend/nonexistent.txt")
print(f"Expected error: {content}")

Expected error: Error: File '/tmp/test_backend/nonexistent.txt' not found


## Test 4: edit() - Edit Files

In [14]:
# Edit the hello.txt file - single replacement
result = backend.edit(file_path="/tmp/test_backend/hello.txt", old_string="Hello World!", new_string="Greetings Universe!", replace_all=False)
print(f"Edit result: {result}")

# Read to verify
content = backend.read("/tmp/test_backend/hello.txt")
print("\nUpdated content:")
print(content)

Edit result: EditResult(error=None, path='/tmp/test_backend/hello.txt', files_update=None, occurrences=1)

Updated content:
     1	Greetings Universe!
     2	This is a test file.
     3	Line 3.


In [15]:
# Edit with replace_all
result = backend.edit(file_path="/tmp/test_backend/test.py", old_string="World", new_string="Runloop", replace_all=True)
print(f"Edit result: {result}")

# Read to verify
content = backend.read("/tmp/test_backend/test.py")
print("\nUpdated content:")
print(content)

Edit result: EditResult(error=None, path='/tmp/test_backend/test.py', files_update=None, occurrences=1)

Updated content:
     1	def greet(name):
     2	    return f'Hello, {name}!'
     3	
     4	if __name__ == '__main__':
     5	    print(greet('Runloop'))


## Test 5: ls_info() - List Directory Contents

In [16]:
# List files in test directory
files = backend.ls_info("/tmp/test_backend")
print(f"Found {len(files)} items:")
for file_info in files:
    print(f"  {file_info}")

Found 2 items:
  {'path': '/tmp/test_backend/hello.txt', 'is_dir': False, 'size': 48, 'modified_at': '2025-11-05T10:00:08.928000'}
  {'path': '/tmp/test_backend/test.py', 'is_dir': False, 'size': 102, 'modified_at': '2025-11-05T10:00:09.276000'}


In [17]:
# Create subdirectories and more files
backend.exec("mkdir -p /tmp/test_backend/subdir")
backend.write("/tmp/test_backend/subdir/nested.txt", "Nested file content")

# List again
files = backend.ls_info("/tmp/test_backend")
print(f"\nAfter adding subdirectory - Found {len(files)} items:")
for file_info in files:
    print(f"  Path: {file_info['path']}, Is Dir: {file_info['is_dir']}, Size: {file_info['size']}")


After adding subdirectory - Found 3 items:
  Path: /tmp/test_backend/hello.txt, Is Dir: False, Size: 48
  Path: /tmp/test_backend/subdir/, Is Dir: True, Size: 0
  Path: /tmp/test_backend/test.py, Is Dir: False, Size: 102


## Test 6: grep_raw() - Search Files

In [18]:
# Create more test files for grepping
backend.write("/tmp/test_backend/file1.txt", "The quick brown fox")
backend.write("/tmp/test_backend/file2.txt", "The lazy dog\nA quick cat")
backend.write("/tmp/test_backend/data.json", '{"key": "value", "number": 42}')

WriteResult(error=None, path='/tmp/test_backend/data.json', files_update=None)

In [19]:
# Search for pattern
matches = backend.grep_raw(pattern="quick", path="/tmp/test_backend")
print(f"Found {len(matches)} matches for 'quick':")
for match in matches:
    print(f"  {match['path']}:{match['line']} - {match['text']}")

Found 2 matches for 'quick':
  /tmp/test_backend/file2.txt:2 - A quick cat
  /tmp/test_backend/file1.txt:1 - The quick brown fox


In [20]:
# Search with glob filter
matches = backend.grep_raw(pattern="quick", path="/tmp/test_backend", glob="*.txt")
print(f"\nFound {len(matches)} matches for 'quick' in *.txt files:")
for match in matches:
    print(f"  {match['path']}:{match['line']} - {match['text']}")


Found 2 matches for 'quick' in *.txt files:
  /tmp/test_backend/file2.txt:2 - A quick cat
  /tmp/test_backend/file1.txt:1 - The quick brown fox


In [21]:
# Search with regex pattern
matches = backend.grep_raw(pattern="[0-9]+", path="/tmp/test_backend")
print(f"\nFound {len(matches)} matches for numbers:")
for match in matches:
    print(f"  {match['path']}:{match['line']} - {match['text']}")


Found 0 matches for numbers:


## Test 7: glob_info() - Find Files by Pattern

In [22]:
# Find all .txt files
files = backend.glob_info(pattern="*.txt", path="/tmp/test_backend")
print(f"Found {len(files)} .txt files:")
for file_info in files:
    print(f"  {file_info['path']} ({file_info['size']} bytes)")

Found 3 .txt files:
  /tmp/test_backend/file1.txt (19 bytes)
  /tmp/test_backend/file2.txt (24 bytes)
  /tmp/test_backend/hello.txt (48 bytes)


In [23]:
# Find all .py files
files = backend.glob_info(pattern="*.py", path="/tmp/test_backend")
print(f"\nFound {len(files)} .py files:")
for file_info in files:
    print(f"  {file_info['path']} ({file_info['size']} bytes)")


Found 1 .py files:
  /tmp/test_backend/test.py (102 bytes)


In [24]:
# Recursive search with **
files = backend.glob_info(pattern="**/*.txt", path="/tmp/test_backend")
print(f"\nFound {len(files)} .txt files (recursive):")
for file_info in files:
    print(f"  {file_info['path']}")


Found 4 .txt files (recursive):
  /tmp/test_backend/file1.txt
  /tmp/test_backend/file2.txt
  /tmp/test_backend/hello.txt
  /tmp/test_backend/subdir/nested.txt


## Cleanup (Optional)

In [ ]:
# Clean up test files
stdout, exit_code = backend.exec("rm -rf /tmp/test_backend")
print(f"Cleanup completed with exit code: {exit_code}")

In [ ]:
# Optionally shutdown the devbox
# client.devboxes.shutdown(id=devbox.id)
# print(f"Devbox {devbox.id} shutdown")

## Summary

This notebook tested all methods of the RunloopProtocol backend:

1. ✓ `exec()` - Execute shell commands
2. ✓ `write()` - Create new files
3. ✓ `read()` - Read file contents with line numbers
4. ✓ `edit()` - Edit files with string replacement
5. ✓ `ls_info()` - List directory contents
6. ✓ `grep_raw()` - Search for patterns in files
7. ✓ `glob_info()` - Find files matching glob patterns